# Ejercicio 04: Evaluación de un Sistema de Recuperación de Información

Integrantes: Rossy Armendariz, Alejandro Chavez, Javier Quishpe.

El objetivo de este ejercicio es evaluar la efectividad de un sistema de recuperación de información utilizando métricas como *precisión*, *recall*, *F1-score*, *Mean Average Precision (MAP)* y *Normalized Discounted Cumulative Gain (nDCG)*.

Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Proporcionar un Conjunto de Datos:
    * Corpus de Documentos: Utiliza el corpus del ejercicio anterior o un nuevo conjunto de documentos.
    * Consultas: Define un conjunto de consultas específicas.
    * Juicios de Relevancia: Proporciona una lista de qué documentos son relevantes para cada consulta.

2. Calcular Resultados de Búsqueda:
    * Obten los resultados ordenados de dos sistemas de recuperación para cada consulta.

3. Calcular las Métricas de Evaluación:
    * Calcular las siguientes métricas para cada sistema y consulta:
        * Precisión en el top-k (Prec@k)
        * Recall
        * F1-score
        * Mean Average Precision (MAP)
        * nDCG

4. Análisis y Comparación:
    * Comparar los resultados de los dos sistemas utilizando las métricas calculadas.
    * Discutir cuál sistema es más efectivo y por qué.

# Proporcionar un Conjunto de Datos
Se utiliza el corpus del ejercicio 01tdmatrix

In [9]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import ndcg_score, average_precision_score

with open("./data/01tdmatrix_corpus.json", "r") as f:
    data = json.load(f)

documentos = {doc["id"]: doc["text"] for doc in data}

Se define un conjunto de consultas específicas.

In [10]:
consultas = [
    "inteligencia artificial en medicina",
    "beneficios de la educación a distancia",
    "realidad aumentada en videojuegos",
    "desarrollo personal y hábitos saludables",
    "futuro del comercio electrónico",
    "tecnologías en cine moderno",
    "competencias de e-sports",
    "diagnóstico con dispositivos portátiles",
    "literatura de ciencia ficción",
    "plataformas de streaming"
]

Se obtiene los juicios de relevancia, es decir, se proporciona una lista de qué documentos son relevantes para cada consulta.

In [11]:
juicios_relevancia = {
    "inteligencia artificial en medicina": [1, 4, 5],
    "beneficios de la educación a distancia": [2, 6, 9],
    "realidad aumentada en videojuegos": [3, 7],
    "desarrollo personal y hábitos saludables": [8, 10],
    "futuro del comercio electrónico": [11, 13, 15],
    "tecnologías en cine moderno": [14, 16],
    "competencias de e-sports": [17, 19],
    "diagnóstico con dispositivos portátiles": [20, 21],
    "literatura de ciencia ficción": [22, 23, 24],
    "plataformas de streaming": [25, 26]
}

# Calcular Resultados de Búsqueda
Se obtiene los resultados ordenados de dos sistemas de recuperación para cada consulta, en este caso se aplicará similitud coseno y jaccard. Primero, se crea la matriz TF-IDF.

In [12]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documentos.values())

Para recuperación basada en la similitud de coseno.

In [13]:
def recuperar_por_coseno(consulta):
    consulta_vec = vectorizer.transform([consulta])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    resultados = [doc_id for doc_id, _ in sorted(zip(documentos.keys(), similitudes), key=lambda x: -x[1])]
    return resultados, similitudes

Para recuperación basada en jaccard.

In [14]:
def recuperar_por_jaccard(consulta):
    consulta_palabras = set(consulta.lower().split())
    resultados = []
    similitudes = []
    for doc_id, doc_text in documentos.items():
        doc_palabras = set(doc_text.lower().split())
        interseccion = consulta_palabras.intersection(doc_palabras)
        union = consulta_palabras.union(doc_palabras)
        jaccard = len(interseccion) / len(union) if len(union) > 0 else 0
        resultados.append(doc_id)
        similitudes.append(jaccard)

    resultados_ordenados = [doc_id for doc_id, _ in sorted(zip(resultados, similitudes), key=lambda x: -x[1])]
    return resultados_ordenados, similitudes

# Calcular las Métricas de Evaluación
Se calcula las siguientes métricas de evaluación para cada sistema y consulta:
- Precisión en el top-k (Prec@k)
- Recall
- F1-score
- Mean Average Precision (MAP)
- nDCG

In [15]:
def calcular_métricas(resultados, relevancia, similitudes):
    k = len(relevancia)
    relevantes_en_top_k = sum([1 for i in resultados[:k] if i in relevancia])
    precision_at_k = relevantes_en_top_k / k
    recall = relevantes_en_top_k / len(relevancia)
    f1 = 2 * (precision_at_k * recall) / (precision_at_k + recall) if (precision_at_k + recall) > 0 else 0
    map_score = average_precision_score([1 if doc in relevancia else 0 for doc in resultados], similitudes)
    ndcg = ndcg_score([relevancia], [resultados[:len(relevancia)]])
    return precision_at_k, recall, f1, map_score, ndcg

# Análisis y Comparación
Se compara los resultados de los dos sistemas utilizando las métricas calculadas.
Discutir cuál sistema es más efectivo y por qué.

In [17]:
metrica_coseno= {}
metrica_jaccard = {}

for consulta in consultas:
    # Recuperación y cálculo de métricas para sistema coseno
    resultados_coseno, similitudes_coseno = recuperar_por_coseno(consulta)
    relevancia = juicios_relevancia[consulta]
    prec_k_cos, recall_cos, f1_cos, map_cos, ndcg_cos = calcular_métricas(resultados_coseno, relevancia, similitudes_coseno)
    metrica_coseno[consulta] = {
        "Prec@k": prec_k_cos, "Recall": recall_cos, "F1-score": f1_cos, "MAP": map_cos, "nDCG": ndcg_cos
    }

    # Recuperación y cálculo de métricas para sistema 2 jaccard
    resultados_jaccard, similitudes_jaccard = recuperar_por_jaccard(consulta)
    prec_k_jac, recall_jac, f1_jac, map_jac, ndcg_jac = calcular_métricas(resultados_jaccard, relevancia, similitudes_jaccard)
    metrica_jaccard[consulta] = {
        "Prec@k": prec_k_jac, "Recall": recall_jac, "F1-score": f1_jac, "MAP": map_jac, "nDCG": ndcg_jac
    }

# Comparación de resultados
for consulta in consultas:
    print(f"Consulta: {consulta}")
    print("Sistema Coseno:", metrica_coseno[consulta])
    print("Sistema Jaccard:", metrica_jaccard[consulta])
    print()

Consulta: inteligencia artificial en medicina
Sistema Coseno: {'Prec@k': 0.0, 'Recall': 0.0, 'F1-score': 0, 'MAP': 0.5694444444444444, 'nDCG': 0.7507390280692656}
Sistema Jaccard: {'Prec@k': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-score': 0.3333333333333333, 'MAP': 0.4444444444444445, 'nDCG': 0.9510464835551906}

Consulta: beneficios de la educación a distancia
Sistema Coseno: {'Prec@k': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-score': 0.3333333333333333, 'MAP': 0.41666666666666663, 'nDCG': 0.9999999999999999}
Sistema Jaccard: {'Prec@k': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-score': 0.3333333333333333, 'MAP': 0.3333333333333333, 'nDCG': 0.9999999999999999}

Consulta: realidad aumentada en videojuegos
Sistema Coseno: {'Prec@k': 0.5, 'Recall': 0.5, 'F1-score': 0.5, 'MAP': 1.0, 'nDCG': 0.9999999999999998}
Sistema Jaccard: {'Prec@k': 0.5, 'Recall': 0.5, 'F1-score': 0.5, 'MAP': 0.625, 'nDCG': 0.9999999999999998}

Consulta: desarrollo personal y háb

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:1030: UserWarning: No positive c

El Sistema de Similitud de Coseno es más efectivo en consultas temáticas generales, mientras que el Sistema de Similitud de Jaccard es mejor para consultas donde la coincidencia exacta de términos es crucial.